#Restful Open Weathermap API

In this notebook, we retrieve the hourly temperatue of san diego over 2013 and plot it.  We use the restful API for openweathermap.org.

In [1]:
%pylab inline
import requests
import datetime 

Populating the interactive namespace from numpy and matplotlib


The [requests](http://docs.python-requests.org/en/latest/) library is a simple to use python library for executing http commands over the internet.  

For example, we can create an http get command to download a website and then render it as html in our notebook.

In [3]:
req = requests.get("http://m.xkcd.com/")
print type(req)
req.cookies?

<class 'requests.models.Response'>


In [6]:
from IPython.display import HTML
HTML(req.text)

Due to the prevelance of HTTP, many webservices now offer [Restful APIs](http://en.wikipedia.org/wiki/Representational_state_transfer) which are accessed via HTTP requests and return XML or JSON formatted data output.  A list of APIs is listed at [Programable Web](http://www.programmableweb.com/).  In additon, many developers of internal apps utilize the restful concept to ease integration of apps.

One easy to understand Restful API is the [Open Weather Map](openweathermap.org) API.  We will use this API to download the daily temperature of San Diego from January 1, 2013 through December 31, 2013 to give an example of the usage of such an API.

##Open Weather Map

As a first step we query the current weather in San Diego to see what the output looks like and get the city id for the historical lookup.  We use the request object's json method to convert the json output to a python Map which we can easily process.

To do this we perform a http get on the URL **http://api.openweathermap.org/data/2.5/weather** which is the base url for the Open Weather Map's current weather resful API.  We pass the get parameter **q=San%20Diego,us** to specify our city.  %20 is used to encode space in the url


In [42]:
req = requests.get("http://api.openweathermap.org/data/2.5/weather?q=San%20Diego,us&&APPID=dd9abf3c45c153e11a6e11f0d12391aa")
output = req.json()
output

{u'base': u'cmc stations',
 u'clouds': {u'all': 40},
 u'cod': 200,
 u'coord': {u'lat': 32.72, u'lon': -117.16},
 u'dt': 1449158160,
 u'id': 5391811,
 u'main': {u'humidity': 33,
  u'pressure': 1019,
  u'temp': 288.75,
  u'temp_max': 292.15,
  u'temp_min': 285.15},
 u'name': u'San Diego',
 u'sys': {u'country': u'US',
  u'id': 473,
  u'message': 0.0052,
  u'sunrise': 1449153301,
  u'sunset': 1449189736,
  u'type': 1},
 u'weather': [{u'description': u'scattered clouds',
   u'icon': u'03d',
   u'id': 802,
   u'main': u'Clouds'}],
 u'wind': {u'deg': 110.002, u'speed': 2.55}}

From here we can see that the city id for san diego (the "id" field) is 5391811.  We can now use open weather map's [Histrocial Data API](http://openweathermap.org/history) to download san diego data.  Lets try pulling the first entry for 2013.  Here are the parameters we pass:

* Base URL: http://api.openweathermap.org/data/2.5/history/city
* id - the city id - 5391811
* start - the unix time stamp of our start period - 1357027200 (0:0:0 on Jan 1 2013)
* cnt - the number of measures we want - 1
* APPID - the *API Key* for david's account - dd9abf3c45c153e11a6e11f0d12391aa


This gives us this URL: **http://api.openweathermap.org/data/2.5/history/city?id=5391811&type=day&start=1357027200&cnt=1&APPID=dd9abf3c45c153e11a6e11f0d12391aa&units=imperial**

In [46]:
req = requests.get("http://api.openweathermap.org/data/2.5/history/city?id=5391811&type=day&start=1449161589&cnt=1&APPID=2c228dcb2a3255a52bde82b5c1d285a8")
output = req.json()
output


<bound method Response.json of <Response [401]>>

From here we can see that the temperature is located at req.json["list"][0]["main"]["temp"].  Interestingly, we also see the temperature is reported in kelvin, which can be converted to Fahrenheit using $F=(K - 273.15)* 1.8000+ 32.00$.


Now we'll loop through from the start period to the end of the year, adding 86400 (the number of seconds in 24 hours) to the time stamp each time.  We'll append the temperature recieved to a list "temp" for us to plot later.

Also note that some times there is no measurement for a day, to do this we check if ["list"] is not empty before extracting main/temp from it.  If it is empty we append None.

In [ ]:
temp = []
time = 1357027200
for i in range(365):
    if i%10 == 0:
        print i
    req = requests.get("http://api.openweathermap.org/data/2.5/history/city?id=5391811&type=day&start=" + str(time) + "&cnt=1&APPID=dd9abf3c45c153e11a6e11f0d12391aa&type=day")
    time = time + 86400
    data = req.json()["list"]
    if len(data) > 0:
        K = data[0]["main"]["temp"]
        F = (K - 273.15)* 1.8000+ 32.00
        temp.append(F)
    else:
        temp.append(None)

In [ ]:
plot(temp)

There is obviously a lot more to learn about Restful API's, however this should provide a simple example of what they are and how to use them.  Explore the programmable web website to see the vast array of api's availible.

**Exercise**
Look at the programmable web website and find an API that interests you.  Use the restful API to query some data and extract a meaningful field.  If the API provides time series Data, plot the field over time.